<a href="https://colab.research.google.com/github/k-washi/tf2-practice/blob/master/tf2-tutorial/tf2_tutorial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [0]:
mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [7]:
print(x_train.shape, type(x_train))

(60000, 28, 28) <class 'numpy.ndarray'>


In [17]:
x_train_n = x_train[..., tf.newaxis]
x_test_n = x_test[..., tf.newaxis]

print(x_train_n.shape, type(x_train))

(60000, 28, 28, 1) <class 'numpy.ndarray'>


In [16]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train_n, y_train)
).shuffle(10000).batch(32)

print(train_ds)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>


In [19]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test_n, y_test)).batch(32)
print(test_ds)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>


In [22]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')
  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)
    
model = MyModel()
print(model)

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
def test_step(images, labels):
  predicts = model(images)
  t_loss = loss_object(labels, predicts)

  test_loss(t_loss)
  test_accuracy(labels, predicts)

In [30]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
  # 次にエポック用にメトリクスをリセット
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.3669925332069397, Accuracy: 86.91166687011719, Test Loss: 0.30109667778015137, Test Accuracy: 88.8499984741211
Epoch 2, Loss: 0.2397753894329071, Accuracy: 91.0183334350586, Test Loss: 0.276358425617218, Test Accuracy: 89.92000579833984
Epoch 3, Loss: 0.18235263228416443, Accuracy: 93.21833038330078, Test Loss: 0.27471888065338135, Test Accuracy: 90.41999816894531
Epoch 4, Loss: 0.14186671376228333, Accuracy: 94.6866683959961, Test Loss: 0.29808539152145386, Test Accuracy: 90.1800003051